In [ ]:
# Install dependencies

import sys
!{sys.executable} -m pip install hdfs

In [1]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/TrabalhoPL/gold'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# create gold database
spark.sql(
    """
    DROP DATABASE IF EXISTS Projeto_gold CASCADE
    """
)
spark.sql(
    """
    create database Projeto_gold location 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db'
    """
)


DataFrame[]

In [9]:
from pyspark.sql.functions import substring, avg, sum

# read recycling from the silver tables
hdfs_path = "hdfs://hdfs-nn:9000/TrabalhoPL/silver/Projeto.db/Tabela_Reciclagem"

recycling = spark\
             .read\
             .load(hdfs_path)

recycling.show()
recycling.printSchema()

+----------+--------+-------------------+----------+--------------------+------------------+----------------+------------------+-----------+
|      Zone|District|Fiscal_Month_Number|Month_Name|Diversion_Rate_Total|Capture_Rate_Paper|Capture_Rate_MGP|Capture_Rate_Total|Fiscal_Year|
+----------+--------+-------------------+----------+--------------------+------------------+----------------+------------------+-----------+
|Brooklyn  |   BKN01|                 10|     April|                13.7|              41.0|            44.3|              42.3|       2016|
|Brooklyn  |   BKN02|                 10|     April|                17.6|              33.2|            55.4|              39.8|       2016|
|Brooklyn  |   BKN03|                 10|     April|                10.9|              30.5|            39.3|              34.1|       2016|
|Brooklyn  |   BKN04|                 10|     April|                13.4|              30.3|            59.3|              42.1|       2016|
|Brooklyn  | 

In [10]:
from pyspark.sql.functions import count
gold_recycling = recycling \
    .groupBy("Zone")\
    .agg(
        sum(recycling.Capture_Rate_Paper).alias("Total_Papel"),
        sum(recycling.Capture_Rate_MGP).alias("Total_Vidro_Plastico")
    ) \
   
gold_recycling.toPandas()


,Zone,Total_Papel,Total_Vidro_Plastico
0,Brooklyn,34107.399984,48414.000059
1,Queens,26562.899977,47233.899937
2,Staten Island,6406.499992,11181.599991
3,Manhattan,23585.999977,35775.500013
4,Bronx,18171.499985,30781.999977


In [11]:
# create recycling table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto_gold.Tabela_Reciclagem
    """
)


spark.sql(
    """
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Reciclagem (
        Zone VARCHAR(250),
        Total_Papel DOUBLE, 
        Total_Vidro_Plastico DOUBLE
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Reciclagem/'
    """
)
       
 

DataFrame[]

In [12]:
# write to delta table
gold_recycling \
    .write  \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Reciclagem/")

In [13]:
# check the results in the table
spark.table("Projeto_gold.Tabela_Reciclagem").toPandas()


,Zone,Total_Papel,Total_Vidro_Plastico
0,Brooklyn,34107.399984,48414.000059
1,Queens,26562.899977,47233.899937
2,Staten Island,6406.499992,11181.599991
3,Manhattan,23585.999977,35775.500013
4,Bronx,18171.499985,30781.999977


In [14]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Reciclagem/`
""").show()

++
||
++
++



In [15]:
spark.sql("""
DROP TABLE IF EXISTS Projeto_gold.Tabela_Reciclagem_Presto
""").show()

spark.sql("""
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Reciclagem_Presto (
        Zone VARCHAR(250),
        Total_Papel DOUBLE, 
        Total_Vidro_Plastico DOUBLE
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Reciclagem/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++

